In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['groq_api_key'] = os.getenv('groq_api_key')


In [8]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [9]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)


In [10]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader = WebBaseLoader('https://pytorch.org/docs/stable/nn.html')
documents = loader.load()

In [13]:
documents

[Document(metadata={'source': 'https://pytorch.org/docs/stable/nn.html', 'title': 'torch.nn — PyTorch 2.6 documentation', 'language': 'en'}, page_content='\n\n\n  \n\n\n\ntorch.nn — PyTorch 2.6 documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Learn\n              \n\n\nGet Started\nRun PyTorch locally or get started quickly with one of the supported cloud platforms\n\n\nTutorials\nWhats new in PyTorch tutorials\n\n\nLearn the Basics\nFamiliarize yourself with PyTorch concepts and modules\n\n\nPyTorch Recipes\nBite-size, ready-to-deploy PyTorch code examples\n\n\nIntro to PyTorch - YouTube Series\nMaster PyTorch basics with our engaging YouTube tutorial series\n\n\n\n\n\n\n\n                Ecosystem\n              \n\n\nTools\nLearn about the tools and frameworks in the PyTorch Ecosystem\n\n\nCommunity\nJoin the PyTorch developer community to contribute, learn, and get your questions answered\n\n\nForums\nA p

In [14]:
text_docs = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=500).split_documents(documents)
text_docs

[Document(metadata={'source': 'https://pytorch.org/docs/stable/nn.html', 'title': 'torch.nn — PyTorch 2.6 documentation', 'language': 'en'}, page_content="torch.nn — PyTorch 2.6 documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Learn\n              \n\n\nGet Started\nRun PyTorch locally or get started quickly with one of the supported cloud platforms\n\n\nTutorials\nWhats new in PyTorch tutorials\n\n\nLearn the Basics\nFamiliarize yourself with PyTorch concepts and modules\n\n\nPyTorch Recipes\nBite-size, ready-to-deploy PyTorch code examples\n\n\nIntro to PyTorch - YouTube Series\nMaster PyTorch basics with our engaging YouTube tutorial series\n\n\n\n\n\n\n\n                Ecosystem\n              \n\n\nTools\nLearn about the tools and frameworks in the PyTorch Ecosystem\n\n\nCommunity\nJoin the PyTorch developer community to contribute, learn, and get your questions answered\n\n\nForums\nA place to discuss 

In [15]:
vector_store = FAISS.from_documents(text_docs, OpenAIEmbeddings())
vector_store

In [16]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C736B2ED90>, search_kwargs={})

In [17]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, 'Containers', 'Search what are Containers in pytorch and what are the types')
retriever_tool.name

'Containers'

In [18]:
tools=[wiki,arxiv,retriever_tool]

In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='Llama-3.3-70b-Versatile')

In [21]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\prati\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [22]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [26]:
from langchain.agents import  AgentExecutor

agent_executor = AgentExecutor(tools=tools, agent=agent, verbose=False)

In [27]:
agent_executor.invoke({
  "input":"Tell me why do we need containers in pytorch and what are the types of containers in pytorch"
})

{'input': 'Tell me why do we need containers in pytorch and what are the types of containers in pytorch',
 'output': 'In PyTorch, containers are used to hold and manage other modules or parameters. They are essential building blocks for creating complex neural network architectures. There are several types of containers in PyTorch, including:\n\n1. **Module**: The base class for all neural network modules.\n2. **Sequential**: A sequential container that holds a sequence of modules.\n3. **ModuleList**: A container that holds a list of modules.\n4. **ModuleDict**: A container that holds a dictionary of modules.\n5. **ParameterList**: A container that holds a list of parameters.\n6. **ParameterDict**: A container that holds a dictionary of parameters.\n\nThese containers can be used to create complex neural network architectures, such as convolutional neural networks (CNNs), recurrent neural networks (RNNs), and transformers. They provide a way to organize and manage the different compone